#A simple example to illustrate how to use qthought

In [1]:
import qthought.utils as ut
from qthought.protocol import Protocol, ProtocolStep
from qthought.quantumsystem import QuantumSystem
from qthought.agents import observe
from qthought.inference_procedures import forward_inference, backward_inference
from projectq.ops import H

## 1. Define protocol steps

In [2]:
def action1(qsys:QuantumSystem):
    H | qsys['s']
    
step1 = ProtocolStep(domain={'Qubit': 's'},
                     descr = 'Prepare Qubit s by applying H ',
                     time  = 0,
                     action = action1)

In [3]:
def action2(qsys:QuantumSystem):
    observe(qsys['Alice_memory'],qsys['s'])
    
step2 = ProtocolStep(domain={'Qubit': 's',
                             'AgentMemory(1)': 'Alice'},
                     descr = 'Alice observes s',
                     time  = 1,
                     action = action2)

In [4]:
def action3(qsys:QuantumSystem):
    observe(qsys['Bob_memory'],qsys['s'])
    
step3 = ProtocolStep(domain={'Qubit': 's',
                             'AgentMemory(1)': 'Bob'},
                     descr = 'Bob observes s',
                     time  = 2,
                     action = action3)

## 2. Assemble protocol

In [5]:
#p = Protocol()
p = step1 + step2 + step3
p

Step 0: Prepare Qubit s by applying H (t:0)
Step 1: Alice observes s(t:1)
Step 2: Bob observes s(t:2)

Requirements: 
------------------------------
Qubit             ['s']
AgentMemory(1)    ['Alice', 'Bob']

## 3. Run protocol

In [6]:
qsys = QuantumSystem(p.get_requirements())
p.run(qsys)

Require Qubit s
Require AgentMemory(1) Alice
Require AgentMemory(1) Bob
0 Prepare Qubit s by applying H  t:0
State:
0.71|000> + 0.71|001>
1 Alice observes s t:1
State:
0.71|000> + 0.71|011>
2 Bob observes s t:2
State:
0.71|000> + 0.71|111>


## 4. Calculate Agent's predictions 

In [7]:
forward_inference(p, 'Alice_memory', 1, 'Bob_memory', 2)

In:(Alice_memory:t1)           |         Out: (Bob_memory:t2)   
---------------------------------------------------------
          0          |         [0]
          1          |         [1]

In [8]:
backward_inference(p, 'Alice_memory', 1, 'Bob_memory', 2)

In:(Bob_memory:t2)   |         Out: (Alice_memory:t1)   
---------------------------------------------------------
          0          |         [0]
          1          |         [1]

In [9]:
qsys

QuantumSystem object: 
Nqubits:      3 
Print order:  ['Bob_memory', 'Alice_memory', 's']
Wavefunction: 
0.71|000> + 0.71|111>